Transformer 是一种革命性的深度学习模型架构，主要用于自然语言处理（NLP）任务。它由Google在2017年的论文《Attention is All You Need》中首次提出。以下是Transformer的核心特点：

1. **自注意力机制（Self-Attention）**：
   - 这是Transformer的核心创新
   - 允许模型在处理每个词时关注输入序列中的所有词
   - 能够捕捉长距离依赖关系

2. **并行计算**：
   - 与RNN不同，Transformer可以并行处理整个序列
   - 大大提高了训练效率

3. **编码器-解码器结构**：
   - 编码器：将输入序列转换为一系列特征表示
   - 解码器：根据编码器的输出生成目标序列

4. **位置编码**：
   - 由于Transformer没有循环结构，需要额外添加位置信息
   - 通过正弦/余弦函数或学习得到的位置编码来实现



Transformer模型可以主要分为以下几个核心部分：

1. **输入部分（Input Processing）**
   - 词嵌入（Word Embedding）
   - 位置编码（Positional Encoding）

2. **编码器部分（Encoder）**
   - 多头自注意力机制（Multi-Head Self-Attention）
   - 前馈神经网络（Feed Forward Network）
   - 残差连接和层归一化（Residual Connection & Layer Normalization）

3. **解码器部分（Decoder）**
   - 掩码多头自注意力机制（Masked Multi-Head Self-Attention）
   - 编码器-解码器注意力机制（Encoder-Decoder Attention）
   - 前馈神经网络（Feed Forward Network）
   - 残差连接和层归一化（Residual Connection & Layer Normalization）

4. **输出部分（Output）**
   - 线性变换（Linear Transformation）
   - Softmax层

5. **辅助组件**
   - 注意力机制（Attention Mechanism）
   - 位置前馈网络（Position-wise Feed Forward Network）
   - 残差连接（Residual Connections）
   - 层归一化（Layer Normalization）

每个部分的具体作用：
- **输入部分**：将离散的单词转换为连续的向量表示，并加入位置信息
- **编码器**：提取输入序列的特征表示
- **解码器**：根据编码器的输出和已生成的部分序列，预测下一个单词
- **输出部分**：将解码器的输出转换为概率分布，用于预测下一个单词
- **辅助组件**：帮助模型更好地训练和收敛

这些部分共同构成了Transformer模型，使其能够有效地处理序列数据，并在各种NLP任务中取得优异的表现。


---

## 1. Input Processing 🐱 输入处理



### 1.1 词嵌入（Word Embedding）



#### 1. **什么是nn.Embedding？**
`nn.Embedding`是PyTorch中的一个模块，用于将离散的整数索引（通常是单词的索引）转换为连续的向量表示。它本质上是一个查找表，其中每个索引对应一个固定大小的向量。

#### 2. **主要参数：**
- `num_embeddings`：词汇表的大小，即有多少个不同的单词
- `embedding_dim`：每个单词向量的维度
- `padding_idx`（可选）：用于指定填充符号的索引，该索引对应的向量不会更新
- `max_norm`（可选）：如果指定，会对向量进行归一化
- `norm_type`（可选）：归一化的类型，默认是L2范数
- `scale_grad_by_freq`（可选）：是否根据词频缩放梯度
- `sparse`（可选）：是否使用稀疏梯度更新



#### 3. **独立使用示例：**


In [1]:
import torch
import torch.nn as nn

# 假设我们有一个词汇表，包含10个单词
vocab_size = 10
# 每个单词用3维向量表示
embedding_dim = 3

# 创建Embedding层
embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)

# 输入是一个包含单词索引的张量
# 例如：[2, 5, 1] 表示一个包含3个单词的句子
input_indices = torch.tensor([2, 5, 1])

# 通过Embedding层获取对应的词向量
output_vectors = embedding(input_indices)

print("输入索引：", input_indices)
print("输出向量：\n", output_vectors)

输入索引： tensor([2, 5, 1])
输出向量：
 tensor([[-0.8799,  0.4084,  0.2450],
        [ 1.1380, -0.1481, -0.4387],
        [ 0.9900,  0.2413, -0.2034]], grad_fn=<EmbeddingBackward0>)


#### 4. **输出的解释**

- 每个单词索引（如2, 5, 1）被转换为一个3维向量
- 这些向量是随机初始化的，可以在训练过程中学习
- `grad_fn`表示这些向量是可训练的，会随着模型训练而更新

#### 5. **实际应用场景：**
- 自然语言处理（NLP）中，用于将单词转换为向量
- 推荐系统中，用于将用户ID或物品ID转换为向量
- 任何需要将离散索引映射到连续向量的场景



### 1.2 位置编码 🐱 Positional Encoding



#### 1. **什么是位置编码？**
位置编码（Positional Encoding）是Transformer模型中用于为输入序列添加位置信息的一种方法。由于Transformer没有像RNN那样的循环结构，它需要额外的机制来理解单词在序列中的位置。

#### 2. **为什么需要位置编码？**
- **Transformer的局限性**：Transformer使用自注意力机制，可以并行处理整个序列，但无法直接获取序列中元素的位置信息
- **保持顺序信息**：自然语言中，单词的顺序非常重要，位置编码帮助模型理解这种顺序
- **捕捉相对位置**：位置编码的设计使得模型能够捕捉到元素之间的相对位置关系

#### 3. **位置编码的公式：**
位置编码使用正弦和余弦函数的组合：
```
PE(pos, 2i) = sin(pos / 10000^(2i/d_model))
PE(pos, 2i+1) = cos(pos / 10000^(2i/d_model))
```
其中：
- `pos`：单词在序列中的位置
- `i`：维度索引
- `d_model`：模型的维度

#### 4. **位置编码的特点：**
- **周期性**：使用正弦和余弦函数，使得编码具有周期性
- **可学习性**：虽然位置编码是固定的，但模型可以通过学习来利用这些信息
- **相对位置**：不同位置之间的编码关系可以帮助模型理解相对位置



#### 5. **独立使用示例：**



In [2]:
import torch
import math

class PositionalEncoding:
    def __init__(self, d_model, max_len=5000):
        self.d_model = d_model
        self.max_len = max_len
        self.pe = self._generate_position_encoding()
        
    def _generate_position_encoding(self):
        position = torch.arange(self.max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, self.d_model, 2) * 
                           -(math.log(10000.0) / self.d_model))
        pe = torch.zeros(self.max_len, self.d_model)
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        return pe.unsqueeze(0)  # (1, max_len, d_model)
    
    def __call__(self, x):
        # x: (batch_size, seq_len, d_model)
        seq_len = x.size(1)
        return x + self.pe[:, :seq_len, :]

# 模型的维度，即每个词向量的长度
# 这个值决定了位置编码和词嵌入的维度
# 通常选择2的幂次方（如16, 32, 64, 128, 256, 512等）
# 较大的维度可以捕捉更丰富的信息，但会增加计算量
d_model = 16

# 最大序列长度，即位置编码支持的最长序列
# 这个值应该大于或等于实际输入序列的最大长度
# 如果输入序列超过这个长度，位置编码将无法正确表示
# 通常设置为一个足够大的值（如100, 200, 512, 1024等）
max_len = 100

# 批量大小，即一次处理的样本数量
# 较大的批量大小可以提高训练效率，但需要更多内存
# 通常根据GPU内存大小和模型复杂度来选择
batch_size = 2

# 序列长度，即每个样本的单词数量
# 这个值应该小于或等于max_len
# 如果序列长度不同，通常需要进行填充或截断
# 在实际应用中，这个值会根据具体任务而变化
seq_len = 10

# 假设我们有一些随机生成的词向量
word_embeddings = torch.randn(batch_size, seq_len, d_model)

# 创建位置编码器
pos_encoder = PositionalEncoding(d_model, max_len)

# 添加位置编码
output = pos_encoder(word_embeddings)

print("原始词向量形状：", word_embeddings.shape)
print("位置编码形状：", pos_encoder.pe.shape)
print("添加位置编码后的形状：", output.shape)

原始词向量形状： torch.Size([2, 10, 16])
位置编码形状： torch.Size([1, 100, 16])
添加位置编码后的形状： torch.Size([2, 10, 16])


#### 6. **输出解释：**

```python
原始词向量形状： torch.Size([2, 10, 16])
位置编码形状： torch.Size([1, 100, 16])
添加位置编码后的形状： torch.Size([2, 10, 16])
```

这些输出形状反映了Transformer模型中输入处理的不同阶段：

1. **原始词向量形状：torch.Size([2, 10, 16])**
   - `2`：批量大小（batch_size），表示同时处理2个样本
   - `10`：序列长度（seq_len），表示每个样本包含10个单词
   - `16`：模型维度（d_model），表示每个单词用16维向量表示

2. **位置编码形状：torch.Size([1, 100, 16])**
   - `1`：表示位置编码是固定的，对所有样本都相同
   - `100`：最大序列长度（max_len），表示位置编码支持的最长序列
   - `16`：模型维度（d_model），与词向量维度一致，方便相加

3. **添加位置编码后的形状：torch.Size([2, 10, 16])**
   - `2`：批量大小保持不变
   - `10`：序列长度保持不变
   - `16`：模型维度保持不变

**维度一致性的原因：**
- 位置编码的维度`[1, 100, 16]`中，`1`表示位置编码是共享的，`100`是预先生成的最大长度，`16`与词向量维度一致
- 在实际使用时，我们只取前`seq_len`个位置编码（`pos_encoder.pe[:, :seq_len, :]`），因此可以与词向量`[2, 10, 16]`直接相加
- 相加操作利用了PyTorch的广播机制，将`[1, 10, 16]`的位置编码广播到`[2, 10, 16]`，与词向量逐元素相加

这种设计确保了：
1. 位置信息能够正确地添加到每个单词的向量表示中
2. 不同样本可以共享相同的位置编码，提高效率
3. 模型能够处理不同长度的序列，只要不超过最大长度`max_len`


In [3]:
import torch
import torch.nn as nn
import math

class TransformerPreprocessor(nn.Module):
    def __init__(self, vocab_size, d_model, max_seq_len):
        super(TransformerPreprocessor, self).__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.position_encoding = PositionalEncoding(d_model, max_seq_len)
        
    def forward(self, x):
        # x: (batch_size, seq_len)
        embeddings = self.embedding(x)  # (batch_size, seq_len, d_model)
        output = self.position_encoding(embeddings)  # (batch_size, seq_len, d_model)
        return output

class PositionalEncoding:
    def __init__(self, d_model, max_len=5000):
        self.d_model = d_model
        self.max_len = max_len
        self.pe = self._generate_position_encoding()
        
    def _generate_position_encoding(self):
        position = torch.arange(self.max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, self.d_model, 2) * 
                           -(math.log(10000.0) / self.d_model))
        pe = torch.zeros(self.max_len, self.d_model)
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        return pe.unsqueeze(0)  # (1, max_len, d_model)
    
    def __call__(self, x):
        # x: (batch_size, seq_len, d_model)
        seq_len = x.size(1)
        return x + self.pe[:, :seq_len, :]

# 使用示例
vocab_size = 10000
d_model = 512
max_seq_len = 100
batch_size = 32
seq_len = 50

preprocessor = TransformerPreprocessor(vocab_size, d_model, max_seq_len)
input_ids = torch.randint(0, vocab_size, (batch_size, seq_len))  # 随机生成输入
output = preprocessor(input_ids)
print(output.shape)  # 输出: torch.Size([32, 50, 512])

torch.Size([32, 50, 512])


---

## 2. Encoder 🐱 编码器

### 2.1 Multi-Head Attention 🐱 多头注意力机制


多头注意力机制通过并行计算多个注意力头，捕捉输入序列中不同子空间的特征。每个注意力头独立计算注意力分数，然后将结果拼接起来，最后通过线性变换得到输出。

多头注意力机制可以分为以下几个关键步骤：
1. 线性变换：将输入映射为查询（Q）、键（K）、值（V）。
2. 分割多头：将Q、K、V拆分为多个注意力头。
3. 计算注意力分数：计算Q和K的点积，并进行缩放和softmax。
4. 加权求和：使用注意力权重对V进行加权求和。
5. 拼接多头：将多个注意力头的输出拼接回原始维度。
6. 线性变换：对拼接后的结果进行线性变换。



#### **2.1.1 线性变换 Q K V**



在多头注意力机制中，**线性变换**是将输入特征映射为查询（Q）、键（K）、值（V）的关键步骤。以下是详细解释：

---

##### 1. **线性变换的定义**
线性变换是通过矩阵乘法将输入特征映射到新的特征空间。具体来说：
- 输入：`x`，形状为`(batch_size, seq_len, d_model)`。
- 输出：`Q`、`K`、`V`，形状仍为`(batch_size, seq_len, d_model)`，但特征表示已经不同。

数学公式：
```python
Q = x · W_Q
K = x · W_K
V = x · W_V
```
其中：
- `W_Q`、`W_K`、`W_V`是可学习的权重矩阵，形状为`(d_model, d_model)`。
- `·`表示矩阵乘法。

---

##### 2. **线性变换的作用**
- **特征空间的转换**：
  - 输入特征`x`可能是词嵌入或位置编码后的表示，这些特征不一定适合直接用于计算注意力分数。
  - 通过线性变换，将`x`映射到更适合计算注意力的特征空间。
- **增加模型的表达能力**：
  - 线性变换引入了可学习的参数，使模型能够根据任务需求动态调整Q、K、V的表示。
  - 这样，模型可以捕捉输入序列中更复杂的依赖关系。
- **分离不同的角色**：
  - Q、K、V在注意力机制中扮演不同的角色：
    - **Q（Query）**：表示当前需要关注的位置。
    - **K（Key）**：表示其他位置的特征，用于与Q计算相似度。
    - **V（Value）**：表示其他位置的实际信息，用于加权求和。
  - 通过独立的线性变换，Q、K、V可以学习到不同的特征表示。



In [4]:
vocab_size = 10000
d_model = 512
max_seq_len = 100
batch_size = 32
seq_len = 50

preprocessor = TransformerPreprocessor(vocab_size, d_model, max_seq_len)
input_ids = torch.randint(0, vocab_size, (batch_size, seq_len))  # 随机生成输入
x = preprocessor(input_ids)
# print("输入 x:\n", x)
print("输入 x 的形状：", x.shape)

输入 x 的形状： torch.Size([32, 50, 512])



我们定义三个线性变换层，分别用于生成Q、K、V：

In [5]:
import torch.nn as nn

query = nn.Linear(d_model, d_model)  # 查询变换
key = nn.Linear(d_model, d_model)    # 键变换
value = nn.Linear(d_model, d_model)  # 值变换



通过线性变换将输入`x`映射为Q、K、V：

In [6]:
Q = query(x)  # (batch_size, seq_len, d_model)
K = key(x)    # (batch_size, seq_len, d_model)
V = value(x)  # (batch_size, seq_len, d_model)

print("Q:\n", Q)
print("Q 的形状：", Q.shape)
print("\n")
print("K:\n", K)
print("K 的形状：", K.shape)
print("\n")
print("V:\n", V)
print("V 的形状：", V.shape)

Q:
 tensor([[[-0.1226,  0.2862,  1.5646,  ..., -0.1917, -0.4041,  0.0341],
         [-0.6776,  1.0134,  0.0759,  ...,  0.0781, -0.3726,  0.6562],
         [-0.4465,  0.3382,  0.7220,  ...,  0.6275, -0.2743,  0.1470],
         ...,
         [ 0.2620,  1.3220, -0.0358,  ..., -0.0144,  0.4962, -0.5854],
         [ 0.8198,  0.8948,  1.6828,  ...,  0.8175,  0.5629,  0.1582],
         [ 0.1410,  0.0686,  1.3047,  ...,  0.1412, -0.5859, -0.4544]],

        [[-0.5151,  0.6531, -0.0987,  ...,  0.1626,  0.1482,  0.3918],
         [-0.6260, -0.0616, -0.0175,  ...,  0.4259, -0.1129,  0.0267],
         [-1.1387,  0.3664,  0.3647,  ...,  0.0475, -0.3409,  0.5844],
         ...,
         [ 1.2401,  0.8850,  0.5799,  ...,  0.6223,  0.4027, -0.3126],
         [ 0.7176,  0.2893,  1.1336,  ...,  0.5507,  0.1798,  0.1274],
         [ 0.1768,  0.4510,  0.1322,  ...,  1.1007, -0.0029, -0.8731]],

        [[-0.7582,  0.9219,  0.3688,  ..., -0.1324, -0.3082,  0.0451],
         [ 0.3861,  0.9874,  0.2233,  ...

---

#### 2.1.2 分割多头 🐱 将 Q K V 分割为多个注意力头



##### 1. **分割多头的目的**
- **并行计算**：通过将Q、K、V拆分为多个注意力头，可以并行计算多个注意力分数，提高计算效率。
- **捕捉不同特征**：每个注意力头可以关注输入序列中的不同子空间，捕捉更丰富的特征。


##### 2. **分割多头的实现**
假设：
- `d_model`：模型维度（例如512，如之前所示）。
- `num_heads`：注意力头的数量（例如16）。
- `head_dim`：每个注意力头的维度（`d_model // num_heads`，例如512 // 16 = 32）。


In [7]:
# Q, K, V 已经通过线性变换生成
batch_size, seq_len, d_model = Q.shape
print(Q.shape)

torch.Size([32, 50, 512])


In [8]:
num_heads = 16
head_dim = d_model // num_heads
# 分割多头：将 d_model 维度拆分为 num_heads * head_dim
#　将`num_heads`维度提到前面，方便后续并行计算。
Q = Q.view(batch_size, seq_len, num_heads, head_dim).transpose(1, 2)  # (batch_size, num_heads, seq_len, head_dim)
K = K.view(batch_size, seq_len, num_heads, head_dim).transpose(1, 2)  # (batch_size, num_heads, seq_len, head_dim)
V = V.view(batch_size, seq_len, num_heads, head_dim).transpose(1, 2)  # (batch_size, num_heads, seq_len, head_dim)

print("Q 的形状：", Q.shape)
print("\n")
print("K 的形状：", K.shape)
print("\n")
print("V 的形状：", V.shape)

Q 的形状： torch.Size([32, 16, 50, 32])


K 的形状： torch.Size([32, 16, 50, 32])


V 的形状： torch.Size([32, 16, 50, 32])




---

##### 3. **代码解释**
1. **`view`操作**：
   - 将`d_model`维度拆分为`num_heads * head_dim`。
   - 例如，如果`d_model=５１２`，`num_heads=１６`，则`head_dim=３２`。
   - 结果形状为`(batch_size, seq_len, num_heads, head_dim)`。

2. **`transpose`操作**：
   - 将`num_heads`维度提到前面，方便后续并行计算。
   - 结果形状为`(batch_size, num_heads, seq_len, head_dim)`。




---

#### 2.1.3 计算注意力分数 🐱 Q 与 K 的点积





##### 1. **计算注意力分数（点积）**

In [9]:
scores = torch.matmul(Q, K.transpose(-2, -1))  # (batch_size, num_heads, seq_len, seq_len)
print("注意力分数 scores 的形状：", scores.shape)


注意力分数 scores 的形状： torch.Size([32, 16, 50, 50])



- **解释**：
  - 计算Q和K的点积，得到注意力分数。
  - `Q`的形状为`(batch_size, num_heads, seq_len, head_dim)`。
  - `K`的形状为`(batch_size, num_heads, seq_len, head_dim)`。
  - `K.transpose(-2, -1)`将K的最后两个维度转置，形状变为`(batch_size, num_heads, head_dim, seq_len)`。
  - 点积结果`score`的形状为`(batch_size, num_heads, seq_len, seq_len)`。



##### **2. 缩放**


In [10]:
scores = scores / torch.sqrt(torch.tensor(head_dim, dtype=torch.float32))
print("缩放后的注意力分数 scores 的形状：", scores.shape)


缩放后的注意力分数 scores 的形状： torch.Size([32, 16, 50, 50])


- **解释**：
  - 使用`sqrt(head_dim)`对点积结果进行缩放。
  - 这是为了防止点积结果过大，导致softmax的梯度消失。



##### **3. Softmax**


In [11]:
import torch.nn.functional as F

attention = F.softmax(scores, dim=-1)  # (batch_size, num_heads, seq_len, seq_len)
print("注意力权重 attention 的形状：", attention.shape)
print("注意力权重 attention 的值：\n", attention)


注意力权重 attention 的形状： torch.Size([32, 16, 50, 50])
注意力权重 attention 的值：
 tensor([[[[0.0207, 0.0130, 0.0080,  ..., 0.0092, 0.0066, 0.0086],
          [0.0213, 0.0136, 0.0079,  ..., 0.0239, 0.0084, 0.0101],
          [0.0152, 0.0163, 0.0103,  ..., 0.0158, 0.0179, 0.0150],
          ...,
          [0.0116, 0.0104, 0.0272,  ..., 0.0093, 0.0225, 0.0113],
          [0.0132, 0.0154, 0.0462,  ..., 0.0072, 0.0119, 0.0136],
          [0.0249, 0.0194, 0.0296,  ..., 0.0206, 0.0259, 0.0189]],

         [[0.0324, 0.0123, 0.0160,  ..., 0.0307, 0.0190, 0.0336],
          [0.0170, 0.0214, 0.0222,  ..., 0.0212, 0.0287, 0.0214],
          [0.0223, 0.0148, 0.0268,  ..., 0.0582, 0.0168, 0.0493],
          ...,
          [0.0206, 0.0195, 0.0137,  ..., 0.0290, 0.0109, 0.0199],
          [0.0225, 0.0074, 0.0186,  ..., 0.0185, 0.0289, 0.0242],
          [0.0128, 0.0104, 0.0125,  ..., 0.0210, 0.0335, 0.0201]],

         [[0.0386, 0.0223, 0.0082,  ..., 0.0065, 0.0319, 0.0141],
          [0.0449, 0.0201, 0.0182,  .

- **解释**：
  - 对最后一个维度（`seq_len`）进行softmax，得到归一化的注意力权重。
  - 注意力权重的形状为`(batch_size, num_heads, seq_len, seq_len)`。

**注意力权重用于衡量输入序列中每个位置对其他位置的重要性，并指导模型如何聚合信息。**

---

```


#### 2.1.4 加权求和 🐱 使用注意力权重对 V 进行加权求和

In [12]:
output = torch.matmul(attention, V)  # (batch_size, num_heads, seq_len, head_dim)
print("加权求和后的输出 output 的形状：", output.shape)
print("加权求和后的输出 output 的值：\n", output)



加权求和后的输出 output 的形状： torch.Size([32, 16, 50, 32])
加权求和后的输出 output 的值：
 tensor([[[[-2.4090e-01, -1.5315e-01, -5.7065e-01,  ...,  1.4542e-02,
           -2.0302e-02, -3.5509e-01],
          [-3.4927e-01, -1.4877e-01, -6.5870e-01,  ..., -1.0107e-01,
           -1.8227e-02, -2.9727e-01],
          [-3.3238e-01, -1.4190e-01, -6.8181e-01,  ..., -1.8562e-01,
           -8.4850e-02, -2.2513e-01],
          ...,
          [-3.5993e-01, -3.1820e-01, -7.4843e-01,  ..., -1.3314e-01,
            7.2016e-02, -2.9883e-01],
          [-3.2962e-01, -1.6897e-01, -6.7369e-01,  ..., -1.1551e-01,
            6.4341e-02, -3.0525e-01],
          [-3.0362e-01, -2.1198e-01, -6.7845e-01,  ..., -1.4262e-01,
            7.0629e-02, -2.7738e-01]],

         [[-4.2497e-01,  3.2842e-01,  6.5800e-02,  ...,  4.9763e-01,
            1.3507e-01,  4.0124e-01],
          [-4.8631e-01,  3.6572e-01,  4.8753e-02,  ...,  5.1702e-01,
            1.3218e-01,  4.5098e-01],
          [-4.7962e-01,  3.5612e-01,  2.6960e-02,  ..., 

---

#### 2.1.5 拼接多头 🐱 将多个注意力头的输出拼接回原始维度


##### **1. 拼接多头的作用**
- **恢复原始维度**：
  - 在分割多头时，我们将`d_model`拆分为`num_heads * head_dim`。
  - 拼接多头的作用是将多个注意力头的输出拼接回`d_model`维度。
- **生成最终输出**：
  - 拼接后的输出形状为`(batch_size, seq_len, d_model)`，可以直接用于后续的计算。




In [13]:
# output 是加权求和的结果，形状为 (batch_size, num_heads, seq_len, head_dim)
batch_size, num_heads, seq_len, head_dim = output.shape
print("output 的形状：", output.shape)

output 的形状： torch.Size([32, 16, 50, 32])


In [14]:
# 1. 转置：将 num_heads 维度移到后面
output = output.transpose(1, 2)  # (batch_size, seq_len, num_heads, head_dim)

# 2. 拼接：将 num_heads 和 head_dim 合并为 d_model
output = output.reshape(batch_size, seq_len, -1)  # (batch_size, seq_len, d_model)

print("拼接后的 output 的形状：", output.shape)


拼接后的 output 的形状： torch.Size([32, 50, 512])


---

#### 2.1.6 线性变换 🐱 将拼接后的输出映射回原始维度

这部分用于将之前获得的拼接结果用线性变换层映射到另外一个特征空间。这也可以用于适应下一部分``Feed-Forward Network``的输入维度。




In [15]:
import torch.nn as nn

# 定义线性变换层
output_projection = nn.Linear(d_model, d_model)

# 线性变换
projected_output = output_projection(output)  # (batch_size, seq_len, d_model)

print("线性变换后的 output 的形状：", projected_output.shape)


线性变换后的 output 的形状： torch.Size([32, 50, 512])


---

### 2.2 Feed-Forward Network 🐱 前馈神经网络


- **特征转换**：
  - 将多头注意力机制的输出进一步映射到更高维的特征空间。
  - 通过非线性激活函数（如ReLU）引入非线性变换。
- **独立处理**：
  - 对序列中的每个位置独立处理，不依赖其他位置的信息。
- **增强表达能力**：
  - 通过多层全连接网络增强模型的表达能力。

前馈神经网络通常由两层全连接层组成：
1. **第一层**：
   - 输入维度：`d_model`
   - 输出维度：`d_ff`（通常为`4 * d_model`）
   - 激活函数：ReLU
2. **第二层**：
   - 输入维度：`d_ff`
   - 输出维度：`d_model`
   - 无激活函数

很像autoencoder的结构不是吗🐱

In [16]:
import torch.nn as nn

class FeedForwardNetwork(nn.Module):
    def __init__(self, d_model, d_ff):
        super(FeedForwardNetwork, self).__init__()
        self.linear1 = nn.Linear(d_model, d_ff)  # 第一层全连接
        self.linear2 = nn.Linear(d_ff, d_model)  # 第二层全连接
        self.activation = nn.ReLU()  # 激活函数

    def forward(self, x):
        # x: (batch_size, seq_len, d_model)
        x = self.linear1(x)  # (batch_size, seq_len, d_ff)
        x = self.activation(x)  # 非线性变换
        x = self.linear2(x)  # (batch_size, seq_len, d_model)
        return x

In [17]:

d_ff = 2048  # 通常为 4 * d_model

# 我们已经获得了projected_output，形状为 (batch_size, seq_len, d_model)
print("projected_output 的形状：", projected_output.shape)
# 前馈神经网络
ffn = FeedForwardNetwork(d_model, d_ff)
ffn_output = ffn(projected_output)

print("ffn_output 的形状：", ffn_output.shape)


projected_output 的形状： torch.Size([32, 50, 512])
ffn_output 的形状： torch.Size([32, 50, 512])


---

### 2.3 Residual Connection & Layer Normalization 🐱 残差连接和层归一化